# PANDAS vs SAS

Para los usuarios potenciales que provienen de SAS, se pretende mostrar cómo se realizarían las diferentes operaciones de SAS en pandas.

General terminology translation

|pandas   |SAS        |
|:-------:|:---------:|
|DataFrame|data set   |
|column   |variable   |
|row      |observation|
|groupby  |BY-group   |
|NaN      |.          |

## Dataframe

Un `DataFrame` en pandas es análogo a un conjunto de datos SAS: una fuente de datos bidimensional con columnas etiquetadas que pueden ser de diferentes tipos. Como se mostrará en este notebook, casi cualquier operación que se pueda aplicar a un conjunto de datos utilizando el paso DATA de SAS, también se puede realizar en pandas.

## Series

Una `Series` es la estructura de datos que representa una columna de un `DataFrame`. SAS no tiene una estructura de datos separada para una sola columna, pero en general, trabajar con una Serie es similar a hacer referencia a una columna en el paso DATOS.

## Index

Cada `DataFrame` y `Series` tiene un `Index`, que son etiquetas en las filas de los datos. SAS no tiene un concepto exactamente análogo. Las filas de un conjunto de datos esencialmente no están etiquetadas, excepto un índice entero implícito al que se puede acceder durante el paso `DATA` (`_N_`).

## Copies vs. in place operations

La mayoría de las operaciones de pandas devuelven copias de Series/DataFrame. Para hacer que los cambios se "mantengan", deberá asignarlos a una nueva variable:

```
sorted_df = df.sort_values("col1")
```

o sobrescribir el original:

```
df = df.sort_values("col1")
```

### Nota

Verá un argumento de palabra clave `inplace=True` disponible para algunos métodos:

```
df.sort_values("col1", inplace=True)
```

**SU USO NO ES RECOMENDADO**

## Constructing a DataFrame from values

```
        data df;
            input x y;
            datalines;
            1 2
            3 4
            5 6
            ;
        run;
```

In [2]:
import pandas as pd
import numpy as np

df = pd.DataFrame({"x": [1, 3, 5], "y": [2, 4, 6]})

df

,x,y
0,1,2
1,3,4
2,5,6


## Reading external data

```
    proc import datafile='tips.csv' dbms=csv out=tips replace;
        getnames=yes;
    run;
```

In [3]:
url = ("https://raw.github.com/pandas-dev/"
"pandas/main/pandas/tests/io/data/csv/tips.csv"
)

tips = pd.read_csv(url)

tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


Al igual que `PROC IMPORT`, `read_csv` puede tomar una serie de parámetros para especificar cómo se deben analizar los datos. Por ejemplo, si los datos estuvieran delimitados por tabuladores y no tuvieran nombres de columna, el comando pandas sería:

```
tips = pd.read_csv("tips.csv", sep="\t", header=None)

# alternatively, read_table is an alias to read_csv with tab delimiter

tips = pd.read_table("tips.csv", header=None)
```

## Limitación de salida

En SAS:

```
    proc print data=df(obs=5);
    run;
```

In [6]:
tips.head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


De forma predeterminada, los pandas truncarán la salida de los marcos de datos grandes para mostrar la primera y la última fila. Esto se puede anular cambiando las opciones de pandas o usando DataFrame.head() o DataFrame.tail()

## Exportando datos

El inverso de `PROC IMPORT` en SAS es `PROC EXPORT`

```
    proc export data=tips outfile='tips2.csv' dbms=csv;
    correr;
```

De manera similar, en pandas, lo opuesto a `read_csv` es `to_csv()`, y otros formatos de datos siguen una API similar.

In [7]:
tips.to_csv(".\\csv\\tips2.csv")

## Data operations

### Operations on columns

```
    data tips;
        set tips;
        total_bill = total_bill - 2;
        new_bill = total_bill / 2;
    run;
```

In [8]:
tips["total_bill"] = tips["total_bill"] - 2

tips["new_bill"] = tips["total_bill"] / 2

tips

,total_bill,tip,sex,smoker,day,time,size,new_bill
0,14.99,1.01,Female,No,Sun,Dinner,2,7.495
1,8.34,1.66,Male,No,Sun,Dinner,3,4.170
2,19.01,3.50,Male,No,Sun,Dinner,3,9.505
3,21.68,3.31,Male,No,Sun,Dinner,2,10.840
4,22.59,3.61,Female,No,Sun,Dinner,4,11.295
...,...,...,...,...,...,...,...,...
239,27.03,5.92,Male,No,Sat,Dinner,3,13.515
240,25.18,2.00,Female,Yes,Sat,Dinner,2,12.590
241,20.67,2.00,Male,Yes,Sat,Dinner,2,10.335
242,15.82,1.75,Male,No,Sat,Dinner,2,7.910


## Filter

El filtrado en SAS se realiza con una instrucción `if` o `where` en una o más columnas.

```
    data tips;
        set tips;
        if total_bill > 10;
    run;

    data tips;
        set tips;
        where total_bill > 10;
        /* equivalent in this case - where happens before the
          DATA step begins and can also be used in PROC statements */
    run;
```

In [9]:
tips[tips["total_bill"] > 10]

,total_bill,tip,sex,smoker,day,time,size,new_bill
0,14.99,1.01,Female,No,Sun,Dinner,2,7.495
2,19.01,3.50,Male,No,Sun,Dinner,3,9.505
3,21.68,3.31,Male,No,Sun,Dinner,2,10.840
4,22.59,3.61,Female,No,Sun,Dinner,4,11.295
5,23.29,4.71,Male,No,Sun,Dinner,4,11.645
...,...,...,...,...,...,...,...,...
239,27.03,5.92,Male,No,Sat,Dinner,3,13.515
240,25.18,2.00,Female,Yes,Sat,Dinner,2,12.590
241,20.67,2.00,Male,Yes,Sat,Dinner,2,10.335
242,15.82,1.75,Male,No,Sat,Dinner,2,7.910


La declaración anterior simplemente pasa una Serie de objetos `True/False` al `DataFrame`, devolviendo todas las filas con `True`.

In [13]:
is_dinner = tips["time"] == "Dinner"

is_dinner

0      True
1      True
2      True
3      True
4      True
       ... 
239    True
240    True
241    True
242    True
243    True
Name: time, Length: 244, dtype: bool

In [14]:
is_dinner.value_counts()

True     176
False     68
Name: time, dtype: int64

In [15]:
tips[is_dinner]

,total_bill,tip,sex,smoker,day,time,size,new_bill
0,14.99,1.01,Female,No,Sun,Dinner,2,7.495
1,8.34,1.66,Male,No,Sun,Dinner,3,4.170
2,19.01,3.50,Male,No,Sun,Dinner,3,9.505
3,21.68,3.31,Male,No,Sun,Dinner,2,10.840
4,22.59,3.61,Female,No,Sun,Dinner,4,11.295
...,...,...,...,...,...,...,...,...
239,27.03,5.92,Male,No,Sat,Dinner,3,13.515
240,25.18,2.00,Female,Yes,Sat,Dinner,2,12.590
241,20.67,2.00,Male,Yes,Sat,Dinner,2,10.335
242,15.82,1.75,Male,No,Sat,Dinner,2,7.910


### `If/then` logic

En SAS, la lógica `if/then` se puede usar para crear nuevas columnas.

```
    data tips;
        set tips;
        format bucket $4.;

        if total_bill < 10 then bucket = 'low';
        else bucket = 'high';
    run;
```


La misma operación en pandas se puede realizar utilizando el método `where` de `numpy`. 

In [16]:
tips["bucket"] = np.where(tips["total_bill"] < 10, "low", "high")

tips

,total_bill,tip,sex,smoker,day,time,size,new_bill,bucket
0,14.99,1.01,Female,No,Sun,Dinner,2,7.495,high
1,8.34,1.66,Male,No,Sun,Dinner,3,4.170,low
2,19.01,3.50,Male,No,Sun,Dinner,3,9.505,high
3,21.68,3.31,Male,No,Sun,Dinner,2,10.840,high
4,22.59,3.61,Female,No,Sun,Dinner,4,11.295,high
...,...,...,...,...,...,...,...,...,...
239,27.03,5.92,Male,No,Sat,Dinner,3,13.515,high
240,25.18,2.00,Female,Yes,Sat,Dinner,2,12.590,high
241,20.67,2.00,Male,Yes,Sat,Dinner,2,10.335,high
242,15.82,1.75,Male,No,Sat,Dinner,2,7.910,high


### Date functionality

SAS proporciona una variedad de funciones para realizar operaciones en columnas de `date/datetime `.

```
    data tips;
        set tips;
        format date1 date2 date1_plusmonth mmddyy10.;
        date1 = mdy(1, 15, 2013);
        date2 = mdy(2, 15, 2015);
        date1_year = year(date1);
        date2_month = month(date2);
        * shift date to beginning of next interval;
        date1_next = intnx('MONTH', date1, 1);
        * count intervals between dates;
        months_between = intck('MONTH', date1, date2);
    run;

```

Las operaciones equivalentes de pandas se muestran a continuación. Además de estas funciones, pandas es compatible con otras funciones de series temporales que no están disponibles en SAS base (como el remuestreo y las compensaciones personalizadas); consulte la documentación de series temporales para obtener más detalles.


In [17]:
tips["date1"] = pd.Timestamp("2013-01-15")

tips["date2"] = pd.Timestamp("2015-02-15")

tips["date1_year"] = tips["date1"].dt.year

tips["date2_month"] = tips["date2"].dt.month

tips["date1_next"] = tips["date1"] + pd.offsets.MonthBegin()

tips["months_between"] = tips["date2"].dt.to_period("M") - tips["date1"].dt.to_period("M")



tips[["date1", "date2", "date1_year", "date2_month", "date1_next", "months_between"]]

,date1,date2,date1_year,date2_month,date1_next,months_between
0,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
1,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
2,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
3,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
4,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
...,...,...,...,...,...,...
239,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
240,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
241,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
242,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>


### Selección de columnas

SAS proporciona palabras clave en el paso DATA para seleccionar, eliminar y cambiar el nombre de las columnas.

```
        data tips;
            set tips;
            keep sex total_bill tip;
        run;

        data tips;
            set tips;
            drop sex;
        run;

        data tips;
            set tips;
            rename total_bill=total_bill_2;
        run;
```
Las mismas operaciones se expresan en pandas a continuación

In [19]:
# Keep certain columns

tips[["sex", "total_bill", "tip"]]

,sex,total_bill,tip
0,Female,14.99,1.01
1,Male,8.34,1.66
2,Male,19.01,3.50
3,Male,21.68,3.31
4,Female,22.59,3.61
...,...,...,...
239,Male,27.03,5.92
240,Female,25.18,2.00
241,Male,20.67,2.00
242,Male,15.82,1.75


In [20]:
# Drop a column

tips.drop("sex", axis=1)

,total_bill,tip,smoker,day,time,size,new_bill,bucket,date1,date2,date1_year,date2_month,date1_next,months_between
0,14.99,1.01,No,Sun,Dinner,2,7.495,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
1,8.34,1.66,No,Sun,Dinner,3,4.170,low,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
2,19.01,3.50,No,Sun,Dinner,3,9.505,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
3,21.68,3.31,No,Sun,Dinner,2,10.840,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
4,22.59,3.61,No,Sun,Dinner,4,11.295,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,27.03,5.92,No,Sat,Dinner,3,13.515,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
240,25.18,2.00,Yes,Sat,Dinner,2,12.590,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
241,20.67,2.00,Yes,Sat,Dinner,2,10.335,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
242,15.82,1.75,No,Sat,Dinner,2,7.910,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>


In [21]:
# Rename a column

tips.rename(columns={"total_bill": "total_bill_2"})

,total_bill_2,tip,sex,smoker,day,time,size,new_bill,bucket,date1,date2,date1_year,date2_month,date1_next,months_between
0,14.99,1.01,Female,No,Sun,Dinner,2,7.495,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
1,8.34,1.66,Male,No,Sun,Dinner,3,4.170,low,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
2,19.01,3.50,Male,No,Sun,Dinner,3,9.505,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
3,21.68,3.31,Male,No,Sun,Dinner,2,10.840,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
4,22.59,3.61,Female,No,Sun,Dinner,4,11.295,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,27.03,5.92,Male,No,Sat,Dinner,3,13.515,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
240,25.18,2.00,Female,Yes,Sat,Dinner,2,12.590,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
241,20.67,2.00,Male,Yes,Sat,Dinner,2,10.335,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
242,15.82,1.75,Male,No,Sat,Dinner,2,7.910,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>


### Sorting by values

Sorting en SAS es realizado via `PROC SORT`

```
    proc sort data=tips;
        by sex total_bill;
    run;
```

In [22]:
tips = tips.sort_values(["sex", "total_bill"])

tips

,total_bill,tip,sex,smoker,day,time,size,new_bill,bucket,date1,date2,date1_year,date2_month,date1_next,months_between
67,1.07,1.00,Female,Yes,Sat,Dinner,1,0.535,low,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
92,3.75,1.00,Female,Yes,Fri,Dinner,2,1.875,low,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
111,5.25,1.00,Female,No,Sat,Dinner,1,2.625,low,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
145,6.35,1.50,Female,No,Thur,Lunch,2,3.175,low,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
135,6.51,1.25,Female,No,Thur,Lunch,2,3.255,low,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,43.35,3.50,Male,Yes,Sun,Dinner,3,21.675,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
156,46.17,5.00,Male,No,Sun,Dinner,6,23.085,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
59,46.27,6.73,Male,No,Sat,Dinner,4,23.135,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
212,46.33,9.00,Male,No,Sat,Dinner,4,23.165,high,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>


## Merging

Las siguientes tablas se utilizarán en los ejemplos de combinación: 

In [23]:
df1 = pd.DataFrame({"key": ["A", "B", "C", "D"], "value": np.random.randn(4)})

df1

,key,value
0,A,-0.858953
1,B,0.663962
2,C,-1.782436
3,D,0.214286


In [24]:
df2 = pd.DataFrame({"key": ["B", "D", "D", "E"], "value": np.random.randn(4)})

df2

,key,value
0,B,-0.884011
1,D,-0.370735
2,D,-1.352258
3,E,1.031428


En SAS, los datos deben ordenarse explícitamente antes de fusionarse. Se logran diferentes tipos de join usando las variables dummy para rastrear si se encontró una coincidencia en uno o ambos frames.

```
    proc sort data=df1;
        by key;
    run;

    proc sort data=df2;
        by key;
    run;

    data left_join inner_join right_join outer_join;
        merge df1(in=a) df2(in=b);

        if a and b then output inner_join;
        if a then output left_join;
        if b then output right_join;
        if a or b then output outer_join;
    run;
```

`pandas DataFrames` tiene un método `merge()`, que proporciona una funcionalidad similar. No es necesario ordenar los datos con anticipación, y los diferentes tipos de combinación se logran a través de la palabra clave how.

In [25]:
inner_join = df1.merge(df2, on=["key"], how="inner")

inner_join

,key,value_x,value_y
0,B,0.663962,-0.884011
1,D,0.214286,-0.370735
2,D,0.214286,-1.352258


In [26]:
left_join = df1.merge(df2, on=["key"], how="left")

left_join

,key,value_x,value_y
0,A,-0.858953,NaN
1,B,0.663962,-0.884011
2,C,-1.782436,NaN
3,D,0.214286,-0.370735
4,D,0.214286,-1.352258


In [27]:
right_join = df1.merge(df2, on=["key"], how="right")

right_join

,key,value_x,value_y
0,B,0.663962,-0.884011
1,D,0.214286,-0.370735
2,D,0.214286,-1.352258
3,E,NaN,1.031428


In [28]:
outer_join = df1.merge(df2, on=["key"], how="outer")

outer_join

,key,value_x,value_y
0,A,-0.858953,NaN
1,B,0.663962,-0.884011
2,C,-1.782436,NaN
3,D,0.214286,-0.370735
4,D,0.214286,-1.352258
5,E,NaN,1.031428


### Group By

El `PROC SUMMARY` de SAS se puede utilizar para agrupar por una o más variables clave y calcular agregaciones en columnas numéricas.

```
    proc summary data=tips nway;
        class sex smoker;
        var total_bill tip;
        output out=tips_summed sum=;
    run;

```

pandas proporciona un mecanismo de agrupación flexible que permite agregaciones similares.

In [29]:
tips_summed = tips.groupby(["sex", "smoker"])[["total_bill", "tip"]].sum()

tips_summed

total_bill     tip
sex    smoker                    
Female No          869.68  149.77
       Yes         527.27   96.74
Male   No         1725.75  302.00
       Yes        1217.07  183.07

## Interoperabilidad de datos

`pandas` proporciona un método `read_sas()` que puede leer datos SAS guardados en formato binario `XPORT` o `SAS7BDAT`.

```
    libname xportout xport 'transport-file.xpt';
    data xportout.tips;
        set tips(rename=(total_bill=tbill));
        * xport variable names limited to 6 characters;
    run;
```

En pandas>

```
df = pd.read_sas("transport-file.xpt")
df = pd.read_sas("binary-file.sas7bdat")
```


# MÁS INFORMACIÓN

Consulte la documentación oficial [aquí](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sas.html#compare-with-sas)